In [4]:
import os
import bz2 
import pandas as pd

In [2]:
path = "../raw_data"

In [8]:
def parse_raw_files(root_path:str, file_path_string:str):
    
    file_path = os.path.join(root_path, file_path_string)

    data = []
    
    with bz2.open(file_path, 'rt', encoding='utf-8') as f:
        for line in f:
            parts = line.strip().split(' ' , 1)
            if len(parts) == 2:
                label, text = parts
                label = label.replace('__label__', '')
                data.append((label, text))
                
    df = pd.DataFrame(data, columns=['label', 'text'])
    df['label'] = df['label'].astype(int)
    
    return df

In [10]:
train_df = parse_raw_files(path,'train.ft.txt.bz2')
train_df.head()

,label,text
0,2,Stuning even for the non-gamer: This sound tra...
1,2,The best soundtrack ever to anything.: I'm rea...
2,2,Amazing!: This soundtrack is my favorite music...
3,2,Excellent Soundtrack: I truly like this soundt...
4,2,"Remember, Pull Your Jaw Off The Floor After He..."


In [9]:
test_df = parse_raw_files(path,'test.ft.txt.bz2')
test_df.head()

,label,text
0,2,Great CD: My lovely Pat has one of the GREAT v...
1,2,One of the best game music soundtracks - for a...
2,1,Batteries died within a year ...: I bought thi...
3,2,"works fine, but Maha Energy is better: Check o..."
4,2,Great for the non-audiophile: Reviewed quite a...
